In [ ]:
!python3 -m pip install pymilvus

# restart the runtime after the installation completes.
# To do so, choose Runtime > Restart runtime from the main menu.

# Use Built-in Roles

Zilliz Cloud offers built-in roles for you to implement flexible role-based access control (RBAC) to your clusters.

## Preparations

You should connect to one of your cluster before you can add built-in roles for users of the cluster.

In [2]:
import json, os, time
from pymilvus import connections, Role, utility

CLUSTER_ENDPOINT="YOUR_CLUSTER_ENDPOINT" # Set your cluster endpoint
TOKEN="YOUR_CLUSTER_TOKEN" # Set your token

connections.connect(
    alias='default', 
    #  Public endpoint obtained from Zilliz Cloud
    uri=CLUSTER_ENDPOINT,
    # API key or a colon-separated cluster username and password
    token=TOKEN, 
)

## Create a user

On Zilliz Cloud console, you can view existing cluster users on the `Users` tab of a cluster. To create a new one, do as follows:

In [3]:
if not 'user1' in utility.list_usernames():
    utility.create_user(user='user1', password='P@ssw0rd!')

You can change the password for the cluster user as follows:

In [4]:
# 2. Update user credentials

utility.update_password(
    user='user1',
    old_password='P@ssw0rd!',
    new_password='P@ssw0rd!!'
)

## List users

As an alternative, you can also list the users of a specific cluster programmatically as follows: 

In [5]:
# 3. List users

# List only names of the cluster users
print(utility.list_usernames())

# List cluster users with their roles
userInfo = utility.list_users(include_role_info=True)

users = [ { "user": u.username, "roles": list(u.roles) } for u in userInfo.groups ]

print(json.dumps(users, indent=4))

['db_admin', 'user1']
[
    {
        "user": "db_admin",
        "roles": [
            "db_admin"
        ]
    },
    {
        "user": "user1",
        "roles": []
    }
]


## Assign roles 

After you have created a new user, you can assign the user some roles. Currently, Zilliz Cloud offers three built-in roles for cluster users, namely cluster administrators (`db_admin`), read-only cluster users (`db_ro`), and cluster users with read and write permissions (`db_rw`). For details on the permissions of these roles, refer to [Manage Built-in Roles](#)

In [6]:
# 4. Assign role

role = Role("db_ro")

role.add_user("user1")

# 5. Get users of a specific role

users = list(role.get_users())

print(json.dumps(users, indent=4))

[
    "user1"
]


You can also remove users from a specific role.

In [ ]:
role.remove_user("user1")

## List roles

You can list all roles with their corresponding cluster users.

In [7]:
# 6. List roles

roleInfo = utility.list_roles(include_user_info=True)

roles = [ { "role": g.role_name, "users": list(g.users) } for g in roleInfo.groups ]

print(json.dumps(roles, indent=4))

[
    {
        "role": "admin",
        "users": [
            "zcloud_root"
        ]
    },
    {
        "role": "db_admin",
        "users": [
            "db_admin"
        ]
    },
    {
        "role": "db_ro",
        "users": [
            "user1"
        ]
    },
    {
        "role": "db_rw",
        "users": []
    },
    {
        "role": "public",
        "users": []
    }
]


## Delete user

Once a cluster user is no longer in need, delete it as follows:

In [8]:
# 7. Delete user
utility.delete_user('user1')